# 7-2. 텍스트 감정분석의 유용성

텍스트 감정분석 접근법</br>
- 기계학습 기반 접근법
- 감성사전 기반 접근법
    - 기계학습 대비 다음의 단점들이 있다.
        1. 분석 대상에 따라 단어의 감성 점수가 달라질 수 있다는 가능성에 대응하기 힘듦
        2. 단순 긍부정을 넘어서 긍부정의 원인이 되는 대상 속성 기반의 감성 분석이 어려움
- 데이터분석 업무 측면에서의 텍스트 분류 모델
    - 일반적인 데이터분석 업무에서는 범주화가 잘된 정형데이터를 필요로 함
        - 정형데이터는 규모가 커질수록 비용이 기하급수적으로 증가할 수 있다.
    - 비정형데이터인 텍스트에 감성분석 기법을 사용
        - 텍스트를 정형데이터로 가공하여 유용한 의사결정 보조자료로 활용 가능함


워드 임베딩(word embedding) 기법</br>
- 단어의 특성을 저차원 벡터 값으로 표현하는 기법
- 머신러닝 기반 감성분석 시 비용을 절감할 수 있음
    - 머신러닝 추론 일치도(정확도)를 크게 향상할 수 있음

# 7-4. 텍스트 데이터의 특징
## (1) 텍스트를 숫자로 표현하는 방법

단어와 그 **단어의 의미를 나타내는 벡터**를 짝지어 보자.</br>
> 마치 사전에서 특정 단어와 그 단어에 대한 설명이 짝지어진 것 처럼

텍스트 데이터를 처리하는 예제
```text
## 텍스트 데이터 예시

    i feel hungry
    i eat lunch
    now i feel happy
```

In [1]:
# 문장 입력 및 공백을 기준으로 split
# 처리해야 할 문장을 파이썬 리스트에 옮겨 담았습니다.
sentences=['i feel hungry', 'i eat lunch', 'now i feel happy']

# 파이썬 split() 메소드를 이용해 단어 단위로 문장을 쪼개 봅니다.
word_list = 'i feel hungry'.split()
print(word_list)

['i', 'feel', 'hungry']


In [2]:
# 텍스트 데이터를 기반으로 사전을 만들기 위한 예시
## python dictionary 활용

index_to_word={}  # 빈 딕셔너리를 만들어서

# 단어들을 하나씩 채워 봅니다. 채우는 순서는 일단 임의로 하였습니다. 그러나 사실 순서는 중요하지 않습니다. 
# <BOS>, <PAD>, <UNK>는 관례적으로 딕셔너리 맨 앞에 넣어줍니다. 
index_to_word[0]='<PAD>'  # 패딩용 단어
index_to_word[1]='<BOS>'  # 문장의 시작지점
index_to_word[2]='<UNK>'  # 사전에 없는(Unknown) 단어
index_to_word[3]='i'
index_to_word[4]='feel'
index_to_word[5]='hungry'
index_to_word[6]='eat'
index_to_word[7]='lunch'
index_to_word[8]='now'
index_to_word[9]='happy'

print(index_to_word)

{0: '<PAD>', 1: '<BOS>', 2: '<UNK>', 3: 'i', 4: 'feel', 5: 'hungry', 6: 'eat', 7: 'lunch', 8: 'now', 9: 'happy'}


In [3]:
# 텍스트 데이터를 숫자로 변환하기
## 예제 기준에서 key와 value를 바꿔버리면 된다.
word_to_index={word:index for index, word in index_to_word.items()}
print(word_to_index)

{'<PAD>': 0, '<BOS>': 1, '<UNK>': 2, 'i': 3, 'feel': 4, 'hungry': 5, 'eat': 6, 'lunch': 7, 'now': 8, 'happy': 9}


In [4]:
# 바뀐 단어의 값(숫자)을 확인
print(word_to_index['feel'])  # 단어 'feel'은 숫자 인덱스 4로 바뀝니다.

4


In [5]:
# 가지고 있는 텍스트 데이터들을 숫자로 바꿔 표현하기
# 문장 1개를 활용할 딕셔너리와 함께 주면, 단어 인덱스 리스트로 변환해 주는 함수를 만들어 봅시다.
# 단, 모든 문장은 <BOS>로 시작하는 것으로 합니다. 
def get_encoded_sentence(sentence, word_to_index):
    return [word_to_index['<BOS>']]+[word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in sentence.split()]

print(get_encoded_sentence('i eat lunch', word_to_index))

[1, 3, 6, 7]


In [6]:
# 텍스트 데이터 사전화 예시
# 여러 개의 문장 리스트를 한꺼번에 숫자 텐서로 encode해 주는 함수입니다. 
def get_encoded_sentences(sentences, word_to_index):
    return [get_encoded_sentence(sentence, word_to_index) for sentence in sentences]

# sentences=['i feel hungry', 'i eat lunch', 'now i feel happy'] 가 아래와 같이 변환됩니다. 
encoded_sentences = get_encoded_sentences(sentences, word_to_index)
print(encoded_sentences)

[[1, 3, 4, 5], [1, 3, 6, 7], [1, 8, 3, 4, 9]]


In [7]:
# 숫자 값으로 표현된 단어들을 다시 원래 단어(sentence)로 복구하기
# 숫자 벡터로 encode된 문장을 원래대로 decode하는 함수입니다. 
def get_decoded_sentence(encoded_sentence, index_to_word):
    return ' '.join(index_to_word[index] if index in index_to_word else '<UNK>' for index in encoded_sentence[1:])  #[1:]를 통해 <BOS>를 제외

print(get_decoded_sentence([1, 3, 4, 5], index_to_word))

i feel hungry


In [8]:
# 문장 encode, decode 예시
# 여러 개의 숫자 벡터로 encode된 문장을 한꺼번에 원래대로 decode하는 함수입니다. 
def get_decoded_sentences(encoded_sentences, index_to_word):
    return [get_decoded_sentence(encoded_sentence, index_to_word) for encoded_sentence in encoded_sentences]

# encoded_sentences=[[1, 3, 4, 5], [1, 3, 6, 7], [1, 8, 3, 4, 9]] 가 아래와 같이 변환됩니다.
print(get_decoded_sentences(encoded_sentences, index_to_word))

['i feel hungry', 'i eat lunch', 'now i feel happy']


# 7-5. 텍스트 데이터의 특징
## (2) Embedding 레이어의 등장
임베딩(Embedding)?
```text
자연어 처리(Natural Language Processing)분야에서 말하는 임베딩(Embedding)
- 사람이 쓰는 자연어를 기께가 이해할 수 있는 숫자형태(vector)로 바꾼 결과
- 혹인 그 일련의 과정 전체
```
What to do with Embedding?
```text
단어나 문장 사이의 코사인 유사도가 가장 높은 단어 구하기 (계산)
단어들 사이의 의미/문법적 정보 도출
단어 사이 문법적 관계 도출 (벡터 간 연산)
```
> 전이 학습(transfer learning) 임베딩
>> 다른 딥러닝 모델의 입력값으로 자주 쓰임
>> 품질 좋은 임베딩을 사용할수록 모델의 성능이 좋아짐
[LINK: 임베딩 레이어를 통해 word to vector](https://wikidocs.net/64779)

In [9]:
# Embedding 레이어를 활용하여 이전 스텝의 텍스트 데이터를 워드 벡터 텐서 형태로 다시 표현
# 아래 코드는 그대로 실행하시면 에러가 발생할 것입니다. 

import numpy as np
import tensorflow as tf
import os

vocab_size = len(word_to_index)  # 위 예시에서 딕셔너리에 포함된 단어 개수는 10
word_vector_dim = 4    # 위 그림과 같이 4차원의 워드 벡터를 가정합니다. 

embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=word_vector_dim, mask_zero=True)

# 숫자로 변환된 텍스트 데이터 [[1, 3, 4, 5], [1, 3, 6, 7], [1, 8, 3, 4, 9]] 에 Embedding 레이어를 적용합니다. 
raw_inputs = np.array(get_encoded_sentences(sentences, word_to_index), dtype='object')
output = embedding(raw_inputs)
print(output)

2023-07-14 06:25:20.697541: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

### Embedding Layer 주의점
#### 위 cell의 코드가 에러난 이유
> Embedding 레이어의 인풋 문장 벡터의 **길이는 일정**해야한다.</br>
>> raw_inputs에 있는 3개 벡터의 길이는 각각 다음과 같다.
>> 4, 4, 5

```text
Tensorflow에서는 다음 함수를 통해 문장 벡터 뒤에 패딩( <PAD> )을 추가하여 길이를 일정하게 맞춰준다.
```
`tf.keras.preprocessing.sequence.pad_sequences`

In [10]:
## tf.keras.preprocessing.sequence.pad_sequences() 코드 예시
raw_inputs = tf.keras.preprocessing.sequence.pad_sequences(raw_inputs,
                                                       value=word_to_index['<PAD>'],
                                                       padding='post',
                                                       maxlen=5)
print(raw_inputs)

## padding 값의 default는 'pre'다.

[[1 3 4 5 0]
 [1 3 6 7 0]
 [1 8 3 4 9]]


`<PAD>`가 0과 매핑되어 있다.

In [11]:
# input 벡터들 (raw_inputs)의 길이를 맞춰주면서 word to vector를 수행하는 코드
vocab_size = len(word_to_index)  # 위 예시에서 딕셔너리에 포함된 단어 개수는 10
word_vector_dim = 4    # 그림과 같이 4차원의 워드 벡터를 가정합니다.

embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=word_vector_dim, mask_zero=True)

# tf.keras.preprocessing.sequence.pad_sequences를 통해 word vector를 모두 일정 길이로 맞춰주어야 
# embedding 레이어의 input이 될 수 있음에 주의해 주세요. 
raw_inputs = np.array(get_encoded_sentences(sentences, word_to_index), dtype=object)
raw_inputs = tf.keras.preprocessing.sequence.pad_sequences(raw_inputs,
                                                       value=word_to_index['<PAD>'],
                                                       padding='post',
                                                       maxlen=5)
output = embedding(raw_inputs)
print(output)

tf.Tensor(
[[[-0.02797605 -0.00805179 -0.00140411 -0.00804634]
  [-0.00691222 -0.01397717  0.00762901  0.01910103]
  [ 0.03633161 -0.03766296 -0.00030882  0.00575482]
  [ 0.00998298 -0.01156216  0.01873604 -0.03365401]
  [-0.01435258 -0.02897682  0.0341973   0.01650895]]

 [[-0.02797605 -0.00805179 -0.00140411 -0.00804634]
  [-0.00691222 -0.01397717  0.00762901  0.01910103]
  [-0.00497545  0.01191349 -0.00841505 -0.02019   ]
  [-0.01158576  0.04504373 -0.02730181 -0.00517433]
  [-0.01435258 -0.02897682  0.0341973   0.01650895]]

 [[-0.02797605 -0.00805179 -0.00140411 -0.00804634]
  [ 0.0422066   0.03172399  0.01955319  0.0169205 ]
  [-0.00691222 -0.01397717  0.00762901  0.01910103]
  [ 0.03633161 -0.03766296 -0.00030882  0.00575482]
  [-0.0476425  -0.01281666  0.01041086  0.02598471]]], shape=(3, 5, 4), dtype=float32)


Q. output의 shape(3, 5, 4)에서 각 값의 의미
>> 3 : 입력문장 개수
>> 5 : 입력문장의 최대 길이
>> 4 : 워드 벡터의 차원 수

# 7-6. 시퀀스 데이터를 다루는 RNN
- 텍스트 데이터를 다루는데 주로 사용되는 딥러닝 모델
    - Recurrent Neural Network (RNN)
    - 시퀀스(Sequence) 형태의 데이터를 처리하기에 최적인 모델로 알려져 있음

[stateful vs stateless 예시; Web의 관점에서](https://www.slideshare.net/xguru/ss-16106464)</br>
[모두의 딥러닝 강좌 - 12강.RNN, 김성훈 교수](https://youtu.be/-SHPG_KMUkQ)

In [12]:
# RNN 모델을 사용하여 텍스트 데이터를 처리하는 예제 코드
## 텍스트 데이터는 이전 cell의 텍스트를 의미
vocab_size = 10  # 어휘 사전의 크기입니다(10개의 단어)
word_vector_dim = 4  # 단어 하나를 표현하는 임베딩 벡터의 차원수입니다. 

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model.add(tf.keras.layers.LSTM(8))   # 가장 널리 쓰이는 RNN인 LSTM 레이어를 사용하였습니다. 이때 LSTM state 벡터의 차원수는 8로 하였습니다. (변경 가능)
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 4)           40        
                                                                 
 lstm (LSTM)                 (None, 8)                 416       
                                                                 
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 537 (2.10 KB)
Trainable params: 537 (2.10 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


> 시퀀스 데이터와 RNN 참고자료

[Youtube Link](https://youtu.be/mG6N0ut9dog?t=1447)

# 7-7. 꼭 RNN이어야 할까?
텍스트 처리에서는 RNN이 아니라 `1-D Convolution Neural Network (1-D CNN)`를 사용할수도 있다.</br>
```text
이전에 이미지 분류기 구현에서 `2-D CNN`을 사용해보았다.

 - 이미지는 시퀀스 데이터가 아니다.

따라서 이미지 분류기 모델에서 사용된 입력은 이미지 전체다.
```
**1-D CNN (with text data)**
- 문장 전체를 한꺼번에 한 방향으로 스캐닝
- 한 번에 처리하는 문장의 양은 7 (len 7)
- 7 단어 이내에서 발견되는 특징을 추출하여 추출결과값을 문장 분류에 사용함

`1-D CNN`방식도 텍스트 처리에서 RNN 못지않은 효율을 보여줌</br>
**CNN 계열의 이점**
- RNN 계열보다 병렬처리가 효율적이다
- 학습 속도가 RNN에 빠르게 진행된다.

In [13]:
# 1-D CNN을 활용하여 텍스트 데이터 처리 및 모델 학습
vocab_size = 10  # 어휘 사전의 크기입니다(10개의 단어)
word_vector_dim = 4   # 단어 하나를 표현하는 임베딩 벡터의 차원 수입니다. 

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model.add(tf.keras.layers.Conv1D(16, 7, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(5))
model.add(tf.keras.layers.Conv1D(16, 7, activation='relu'))
model.add(tf.keras.layers.GlobalMaxPooling1D())
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 4)           40        
                                                                 
 conv1d (Conv1D)             (None, None, 16)          464       
                                                                 
 max_pooling1d (MaxPooling1  (None, None, 16)          0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, None, 16)          1808      
                                                                 
 global_max_pooling1d (Glob  (None, 16)                0         
 alMaxPooling1D)                                                 
                                                                 
 dense_2 (Dense)             (None, 8)                

`1-D CNN` 기법</br>
- `GlobalMaxPooling()` 레이어만 사용하여 모델을 쌓는 방식

**GlobalMaxPooling** 방식</br>
- 전체 문장 중에서 단 하나의 가장 중요한 단어만 feature로 추출하는 방식
    - 추출된 feature는 문장의 긍정/부정을 평가하는데 사용된다.

In [14]:
# 1-D CNN 기법에서 GlobalMaxPooling() 레이어만 사용하는 예제

vocab_size = 10  # 어휘 사전의 크기입니다(10개의 단어)
word_vector_dim = 4   # 단어 하나를 표현하는 임베딩 벡터의 차원 수입니다. 

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model.add(tf.keras.layers.GlobalMaxPooling1D())
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 4)           40        
                                                                 
 global_max_pooling1d_1 (Gl  (None, 4)                 0         
 obalMaxPooling1D)                                               
                                                                 
 dense_4 (Dense)             (None, 8)                 40        
                                                                 
 dense_5 (Dense)             (None, 1)                 9         
                                                                 
Total params: 89 (356.00 Byte)
Trainable params: 89 (356.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


> 이 외에도 고려할만한 방법

- 1-D CNN과 RNN 섞어 쓰기
- FFN (FeedForward Network) 레이어만 사용해보기
- Transformer 레이어 사용해보기

[참고링크](https://wikidocs.net/80437)

# 7-8. IMDB 영화리뷰 감성분석
## (1) IMDB 데이터셋 분석
IMDb 영화리뷰 감성분석 태스크 도전 예제</br>
**IMDb Large Movie Dataset**</br>
- 50000개의 영단어로 작성된 영화 리뷰 텍스트
- 긍정 리뷰에는 `1`, 부정 리뷰에는 `0`의 라벨이 달려 있다.

[관련논문](https://aclanthology.org/P11-1015.pdf)</br>

> 아래는 노드에 언급된 내용
```text
50000개의 리뷰 중 절반인 25000개가 훈련용 데이터, 나머지 25000개를 테스트용 데이터로 사용하도록 지정되어 있습니다. 이 데이터셋은 tensorflow Keras 데이터셋 안에 포함되어 있어서 손쉽게 다운로드하여 사용할 수 있습니다.
이후 스텝의 IMDb 데이터셋 처리 코드 중 일부는 Tensorflow 튜토리얼에 언급된 데이터 전처리 로직을 참고하였음을 밝힙니다.
```

In [15]:
# IMDb 데이터셋 다운로드
imdb = tf.keras.datasets.imdb

# 학습 데이터 (train)와 정답 데이터(test)로 분리
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)
print(f"훈련 샘플 개수: {len(x_train)}, 테스트 개수: {len(x_test)}")

훈련 샘플 개수: 25000, 테스트 개수: 25000


In [16]:
# 데이터의 예시 확인
print(x_train[0])  # 1번째 리뷰데이터
print('라벨: ', y_train[0])  # 1번째 리뷰데이터의 라벨
print('1번째 리뷰 문장 길이: ', len(x_train[0]))
print('2번째 리뷰 문장 길이: ', len(x_train[1]))

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
라벨:  1
1번째 리뷰 문장 길이:  218
2번째 리뷰 문장 길이:  189


**IMDb 데이터셋 특징**</br>
- 텍스트 데이터가 아니다
- 각 텍스트(단어)들은 이미 숫자로 `encode`되어 있다.
    - 이라한 특징 때문에 IMDb 데이터셋은 encode에 사용한 사전(딕셔너리)까지 함께 제공함

In [17]:
# IMDb 데이터셋의 텍스트 데이터를 가져옴;
## get_word_index()를 사용하여 이미 숫자로 encode된 텍스트 데이터를 읽어오게 됨
word_to_index = imdb.get_word_index()
## 읽어온 데이터(word_to_index)를 숫자키(인덱스)로도 읽을 수 있게 python dictionary 재구성 
index_to_word = {index:word for word, index in word_to_index.items()}
print(index_to_word[1])     # 'the' 가 출력됩니다. 
print(word_to_index['the'])  # 1 이 출력됩니다.

the
1


> 노드에서 언급한 주의점

```text
여기서 주의할 점이 있습니다. IMDb 데이터셋의 텍스트 인코딩을 위한 word_to_index, index_to_word는 보정이 필요한데요.

예를 들어 다음 코드를 실행시켜보면 보정이 되지 않은 상태라 문장이 이상함을 확인하실 겁니다. (뒤에 보정 후 다시 확인해볼 예정이에요.😊)
```

IMDB 데이터셋을 텍스트 인코딩 하기 위해서 `word_to_index`, `index_to_word`들에 대해 보정이 필요함

In [18]:
# 보정 전 x_train[0] 데이터
print(get_decoded_sentence(x_train[0], index_to_word))

as you with out themselves powerful lets loves their becomes reaching had journalist of lot from anyone to have after out atmosphere never more room and it so heart shows to years of every never going and help moments or of every chest visual movie except her was several of enough more with is now current film as you of mine potentially unfortunately of you than him that with out themselves her get for was camp of you movie sometimes movie that with scary but and to story wonderful that in seeing in character to of 70s musicians with heart had shadows they of here that with her serious to have does when from why what have critics they is you that isn't one will very to as itself with other and in of seen over landed for anyone of and br show's to whether from than out themselves history he name half some br of and odd was two most of mean for 1 any an boat she he should is thought frog but of script you not while history he heart to real at barrel but when from one bit then have two of

In [19]:
## x_train 데이터 보정 작업 예시
#실제 인코딩 인덱스는 제공된 word_to_index에서 index 기준으로 3씩 뒤로 밀려 있습니다.  
word_to_index = {k:(v+3) for k,v in word_to_index.items()}

# 처음 몇 개 인덱스는 사전에 정의되어 있습니다.
word_to_index["<PAD>"] = 0      # padding
word_to_index["<BOS>"] = 1      # begining of sentence
word_to_index["<UNK>"] = 2      # unknown
word_to_index["<UNUSED>"] = 3   # unused

index_to_word = {index:word for word, index in word_to_index.items()}

print(index_to_word[1])     # '<BOS>' 가 출력됩니다. 
print(word_to_index['the'])  # 4 이 출력됩니다. 
print(index_to_word[4])     # 'the' 가 출력됩니다.

# 보정 후 x_train[0] 데이터
print(get_decoded_sentence(x_train[0], index_to_word))

<BOS>
4
the
this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for

In [20]:
# encode 된 데이터를 decode 하여 결과 확인하는 예시
print(get_decoded_sentence(x_train[0], index_to_word))
print('라벨: ', y_train[0])  # 1번째 리뷰데이터의 라벨

this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they h

> 노드에서 언급할 주의점

```text
pad_sequences를 통해 데이터셋 상의 문장의 길이를 통일하는 것을 잊어서는 안됩니다.
문장 최대 길이 maxlen의 값 설정도 전체 모델 성능에 영향을 미치게 됩니다. 이 길이도 적절한 값을 찾기 위해서는 전체 데이터셋의 분포를 확인해 보는 것이 좋습니다.
```
>> `pad_sequences`: 데이터셋 상의 문장 길이를 통일
>> 문장 최대 길이 (`maxlen`의 값) 설정도 전체 모델 성능에 영향을 미침

In [21]:
# 전체 데이터셋 확인예시

total_data_text = list(x_train) + list(x_test)
# 텍스트데이터 문장길이의 리스트를 생성한 후
num_tokens = [len(tokens) for tokens in total_data_text]
num_tokens = np.array(num_tokens)
# 문장길이의 평균값, 최대값, 표준편차를 계산해 본다. 
print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))

# 예를들어, 최대 길이를 (평균 + 2*표준편차)로 한다면,  
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
maxlen = int(max_tokens)
print('pad_sequences maxlen : ', maxlen)
print(f'전체 문장의 {np.sum(num_tokens < max_tokens) / len(num_tokens)}%가 maxlen 설정값 이내에 포함됩니다. ')

문장길이 평균 :  234.75892
문장길이 최대 :  2494
문장길이 표준편차 :  172.91149458735703
pad_sequences maxlen :  580
전체 문장의 0.94536%가 maxlen 설정값 이내에 포함됩니다. 


> 노드에서 언급한 내용

```text
위의 경우에는 maxlen=580이 됩니다.
또 한 가지 유의해야 하는 것은 padding 방식을 문장 뒤쪽('post')과 앞쪽('pre') 중 어느 쪽으로 하느냐에 따라 RNN을 이용한 딥러닝 적용 시 성능 차이가 발생한다는 점입니다.
두 가지 방식을 한 번씩 다 적용해서 RNN을 학습시켜 보면서 그 결과를 비교해 보시기 바랍니다.
```
> 벡터 길이 맞출 때, padding 방식에는 두 가지 방식이 있다.

1. 길이를 맞출 때, padding 값을 뒤쪽에 삽입하는 방식: `post`
2. 길이를 맞출 때, padding 값을 앞쪽에 삽입하는 방식: `pre`

예제([Link](https://wikidocs.net/24586))를 사용하여 두 가지의 함수를 선언 및 테스트 수행</br>

> post_rnn_test()
>> post-padding으로 pre-processing한 데이터 기반 RNN 학습
> pre_rnn_test()
>> pre-padding으로 pre-processing한 데이터 기반 RNN 학습

In [ ]:
# POST PADDING 방식의 rnn 실행 예제 함수 선언
def post_rnn_test(x_train, x_test, y_train, y_test, maxlen):
    # 예제에서 사용한 모듈
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, GRU, Embedding
    from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
    from tensorflow.keras.models import load_model
    # 예제 값 그대로 사용
    vocab_size = 10000
    # max_len은 노드 순서상 선언이 되어 있으므로 가져와서 사용함
    # 아래는 샘플 값
    ## max_len = 500

    # POST 패딩; 구성은 노드 참고함
    X_train = tf.keras.preprocessing.sequence.pad_sequences(x_train,
                                                        value=word_to_index["<PAD>"],
                                                        padding='post', # 혹은 'pre'
                                                        maxlen=maxlen)
    X_test = tf.keras.preprocessing.sequence.pad_sequences(x_test,
                                                       value=word_to_index["<PAD>"],
                                                       padding='post', # 혹은 'pre'
                                                       maxlen=maxlen)
    # 공식사이트 기준 하이퍼 파라미터
    ## 하이퍼파라미터인 임베딩 벡터의 차원은 100, 은닉 상태의 크기는 128입니다.
    embedding_dim = 100
    hidden_units = 128

    # Sequential 모델 적용
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim))
    model.add(GRU(hidden_units))
    model.add(Dense(1, activation='sigmoid'))

    
    ## EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)는
    ## 검증 데이터 손실(val_loss)이 증가하면, 과적합 징후므로 검증 데이터 손실이 4회 증가하면
    ## 정해진 에포크가 도달하지 못하였더라도 학습을 조기 종료(Early Stopping)한다는 의미입니다.
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
    ## ModelCheckpoint를 사용하여 검증 데이터의 정확도(val_acc)가 이전보다 좋아질 경우에만 모델을 저장합니다.
    mc = ModelCheckpoint('GRU_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
    ## validation_split=0.2을 사용하여 훈련 데이터의 20%를 검증 데이터로 분리해서 사용하고,
    ## 검증 데이터를 통해서 훈련이 적절히 되고 있는지 확인합니다.
    ## 검증 데이터는 기계가 훈련 데이터에 과적합되고 있지는 않은지 확인하기 위한 용도로 사용됩니다.
    history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)
    
    # GRU 모델을 불러 검증데이터의 정확도 확인
    ## 훈련 과정에서 검증 데이터의 정확도가 가장 높았을 때 저장된 모델인 'GRU_model.h5'를 로드합니다.
    loaded_model = load_model('GRU_model.h5')
    print("\n post-padding 기반 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

# PRE PADDING 방식의 rnn 실행 예제 함수 선언
def pre_rnn_test(x_train, x_test, y_train, y_test, maxlen):
    # 예제에서 사용한 모듈
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, GRU, Embedding
    from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
    from tensorflow.keras.models import load_model
    # 예제 값 그대로 사용
    vocab_size = 10000
    # max_len은 노드 순서상 선언이 되어 있으므로 가져와서 사용함
    # 아래는 샘플 값
    ## max_len = 500

    # POST 패딩; 구성은 노드 참고함
    X_train = tf.keras.preprocessing.sequence.pad_sequences(x_train,
                                                        value=word_to_index["<PAD>"],
                                                        padding='pre', # 혹은 'post'
                                                        maxlen=maxlen)
    X_test = tf.keras.preprocessing.sequence.pad_sequences(x_test,
                                                       value=word_to_index["<PAD>"],
                                                       padding='pre', # 혹은 'post'
                                                       maxlen=maxlen)
    # 공식사이트 기준 하이퍼 파라미터
    ## 하이퍼파라미터인 임베딩 벡터의 차원은 100, 은닉 상태의 크기는 128입니다.
    embedding_dim = 100
    hidden_units = 128

    # Sequential 모델 적용
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim))
    model.add(GRU(hidden_units))
    model.add(Dense(1, activation='sigmoid'))

    
    ## EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)는
    ## 검증 데이터 손실(val_loss)이 증가하면, 과적합 징후므로 검증 데이터 손실이 4회 증가하면
    ## 정해진 에포크가 도달하지 못하였더라도 학습을 조기 종료(Early Stopping)한다는 의미입니다.
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
    ## ModelCheckpoint를 사용하여 검증 데이터의 정확도(val_acc)가 이전보다 좋아질 경우에만 모델을 저장합니다.
    mc = ModelCheckpoint('GRU_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
    ## validation_split=0.2을 사용하여 훈련 데이터의 20%를 검증 데이터로 분리해서 사용하고,
    ## 검증 데이터를 통해서 훈련이 적절히 되고 있는지 확인합니다.
    ## 검증 데이터는 기계가 훈련 데이터에 과적합되고 있지는 않은지 확인하기 위한 용도로 사용됩니다.
    history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)
    
    # GRU 모델을 불러 검증데이터의 정확도 확인
    ## 훈련 과정에서 검증 데이터의 정확도가 가장 높았을 때 저장된 모델인 'GRU_model.h5'를 로드합니다.
    loaded_model = load_model('GRU_model.h5')
    print("\n pre-padding 기반 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

# 함수 실행1
print("post padding case\n ===========")
post_rnn_test(x_train, x_test, y_train, y_test, maxlen)

# 함수 실행2
print("pre padding case\n ===========")
pre_rnn_test(x_train, x_test, y_train, y_test, maxlen)

## 위 코드를 로컬(MAC)에서 검증할 경우, kernel truncated 발생
### why? CPU default calucation
#### 노드에서 확인한 결과 캡처본으로 대체
> post-padding 전처리 테스트 결과 예시

![post-padding example](../images/1.png)</br>

> pre-padding 전처리 테스트 결과 예시

![pre-padding example](../images/2.png)</br>

아래 코드는 노드(`7-8`)에서 실제로 처리해준 과정</br>


In [23]:
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train,
                                                        value=word_to_index["<PAD>"], 
                                                        padding='post', # 혹은 'pre'
                                                        maxlen=maxlen)

x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test,
                                                        value=word_to_index["<PAD>"], 
                                                        padding='post', # 혹은 'pre'
                                                        maxlen=maxlen)

노드에서 언급된 질문</br>
- Q. RNN 활용 시 pad_sequences의 padding 방식은 'post'와 'pre' 중 어느 것이 유리할까요? 그 이유는 무엇일까요?

예시 답변</br>
- RNN은 입력데이터가 순차적으로 처리되어, 가장 마지막 입력이 최종 state 값에 가장 영향을 많이 미치게 됩니다. 그러므로 마지막 입력이 무의미한 padding으로 채워지는 것은 비효율적입니다. 따라서 'pre'가 훨씬 유리하며, 10% 이상의 테스트 성능 차이를 보이게 됩니다.

# 7-9. IMDB 영화리뷰 감상분석
## (2) 딥러닝 모델 설계와 훈련

RNN 모델 직접 설계해보기

In [24]:
vocab_size = 10000  # 어휘 사전의 크기입니다(10,000개의 단어)
word_vector_dim = 16   # 단어 하나를 표현하는 임베딩 벡터의 차원 수입니다. 

# model 설계 - 딥러닝 모델 코드를 직접 작성해 주세요
model = tf.keras.Sequential()
# [[YOUR CODE]]
model.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))  # Embedding Layer
model.add(tf.keras.layers.LSTM(10)) # One of the famouse RNN models; 이전 실습과 다르게 10-dimension 설정해봄
model.add(tf.keras.layers.Dense(16, activation='relu')) # output shape를 16으로 설정해봄
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))   # 최종출력은 긍정 / 부정을 표현하는 1-dimension

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, None, 16)          160000    
                                                                 
 lstm_1 (LSTM)               (None, 10)                1080      
                                                                 
 dense_6 (Dense)             (None, 16)                176       
                                                                 
 dense_7 (Dense)             (None, 1)                 17        
                                                                 
Total params: 161273 (629.97 KB)
Trainable params: 161273 (629.97 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


model 훈련전에 다음을 세팅함</br>
- 훈련용 데이터셋 25,000건
    - 이 중 10,000건을 분리
        - 해당 데이터는 검증셋 (validation set)으로 사용

In [25]:
# validation set 10000건 분리
x_val = x_train[:10000]
y_val = y_train[:10000]

# validation set을 제외한 나머지 15000건
partial_x_train = x_train[10000:]
partial_y_train = y_train[10000:]

print(partial_x_train.shape)
print(partial_y_train.shape)

(15000, 580)
(15000,)


model 학습을 시작해 봅시다.</br>
> 이전 과정에서 데이터들(`x_train`, `x_test`)에 대해 padding 전처리를 하지 않으면 Numpy Error (차원 불일치)가 발생한다.

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

epochs=20   # 얼마나 훈련하면 좋을지 결과를 보면서 바꾸어보기

history = model.fit(partial_x_train, partial_y_train, epochs=epochs, batch_size=512, validation_data=(x_val, y_val), verbose=1)

로컬 테스트 (only MAC) 경우 Truncated 발생</br>
이후 과정은 노드 캡처본으로 정리 대체함</br>

![20 epoch learning example](../images/3.png)</br>

학습이 끝난 모델을 테스트셋으로 평가해 봅시다.

In [ ]:
results = model.evaluate(x_text, y_test, verbose=2)

print(results)

![evaluation with test set](../images/4.png)</br>

`model.fit()` 과정 중의 train/validation loss, accuracy 등이 매 훈련(`epoch`)마다 history 변수에 저장되어 있습니다.</br>
이 데이터를 그래프로 그려 보면, 수행했던 딥러닝 학습이 잘 진행되었는지, 잘못되었는지 (`overfitting` 혹은 `underfitting`), 성능을 개선할 수 있는 다양한 아이디어를 얻을 수 있다.

In [ ]:
history_dict = history.history
print(history_dict.keys())  # epoch에 따른 그래프를 그려볼 수 있는 항목들

![check dictionary key of 'history'](../images/5.png)</br>

In [ ]:
import matplotlib.pyplot as plt

acc = history_dict['accuracy']
val_acc = histroy_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "bo"는 "파란색 점"입니다.
plt.plot(epochs, loss, 'bo', label='Training loss')
# b는 "파란 실선"입니다.
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

![graph of the Training and validation loss](../images/6.png)</br>

validation loss의 그래프가 train loss와의 이격(벌어짐 현상)이 발생하게 되면 더 이상의 트레이닝은 무의미해지게 된다고 한다.

In [ ]:
plt.clf()   # 그림을 초기화합니다.

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

![graph of the Training and validation accuracy](../images/7.png)</br>

# 7-10. IMDB 영화리뷰 감성분석
## (3) Word2Vec의 적용

여기부터 워드 벡터 파일을 저장할 디렉토리를 설정한다.</br>

```shell
# 노드 예시
$ mkdir -p ~/aiffel/sentiment_classification/data
$ pip list | grep gensim

# Local dir example --> ~/data/sentiment_classification
```

> 테스트 기준 gensim version is `4.1.2`

[Youtube: 딥러닝 자연어처리](https://youtube.com/watch?v=sY4YyacSsLc&t=126s&ab_channel=MinsukHeo허민석)

In [28]:
embedding_layer = model.layers[0]
weights = embedding_layer.get_weights()[0]
print(weights.shape)    # shape format: (vocab_size, embedding_dim)

(10000, 16)


In [29]:
# 학습한 Embedding 파라미터를 파일에 써서 저장합니다.
# 노드 기준 path
# word2vec_file_path = os.getenv('HOME')+'/aiffel/sentiment_classification/data/word2vec.txt'
# 로컬 기준 path
word2vec_file_path = '../data/sentiment_classification/word2vec.txt'
f = open(word2vec_file_path, 'w')
# 백터 개수와 사이즈를 어떻게 기재할지 타이틀 작성
f.write('{} {}\n'.format(vocab_size-4, word_vector_dim))

# 단어 개수만큼의 워드 벡터를 파일에 기록합니다.
## 이때, 특수문자 4개는 제외시킵니다.
vectors = model.get_weights()[0]
for i in range(4,vocab_size):
    f.write('{} {}\n'.format(index_to_word[i], ' '.join(map(str, list(vectors[i, :])))))
f.close()

`gensim` 패키지를 이용해, 위에서 작성한 파일(`임베딩 파라미터`)을 읽어서 word vector로 활용할 수 있습니다.

In [30]:
from gensim.models.keyedvectors import Word2VecKeyedVectors

word_vectors = Word2VecKeyedVectors.load_word2vec_format(word2vec_file_path, binary=False)
vector = word_vectors['computer']
vector

array([-0.07126842,  0.08547736,  0.02286462, -0.03098332,  0.0765068 ,
       -0.01164088,  0.060796  , -0.02766533,  0.08184147, -0.06716599,
       -0.01517043,  0.02369968,  0.0872837 , -0.02541867, -0.0553827 ,
       -0.05494674], dtype=float32)

위의 워드 벡터로 실험을 할 수 있다.</br>

> 워드 벡터가 의미 벡터 공간상에 유의미하게 학습되었는지 확인하는 방법 중 하나
>> 단어를 하나 주고 그와 가장 유사한 단어와 유사도를 확인하는 방법

In [31]:
# gensim 패키지를 사용하면 아래와 같이 해볼 수 있음
word_vectors.similar_by_word("love")

[('finely', 0.9213090538978577),
 ('reality', 0.9122799634933472),
 ('plant', 0.905373752117157),
 ('1st', 0.8927375674247742),
 ('relatively', 0.8903787136077881),
 ('portrayal', 0.8892521262168884),
 ('ethan', 0.8865978717803955),
 ('four', 0.8793693780899048),
 ('pleasing', 0.8763188123703003),
 ('toy', 0.8724988102912903)]

**Word2Vec**</br>

구글에서 제공하는 사전학습된(Pretrained) 워드 임베딩 모델
- 1억 개의 단어로 구성된 Google News dataset을 바탕으로 학습된 모델
- 총 300만 개의 단어를 각각 300차원의 벡터로 표현함

임베딩의 개념에 대한 정리 (`한국어 임베딩`)</br>

- [한국어 임베딩 서문](https://ratsgo.github.io/natural%20language%20processing/2019/09/12/embedding/)

---

Word2Vec 모델 적용해보기</br>
> 노드에서는 데이터 다운로드 없이 심볼릭 링크 활용함
```shell
$ ln -s ~/data/GoogleNew-vectors-negative300.bin.gz ~/aiffel/sentiment_classification/data
```

[파일 다운로드 링크](https://kaggle.com/datasets/leadbest/googlenewsvectorsnegative300)</br>
> 2023.07.14 기준 파일 크기가 `3.64 GB`다.

In [ ]:
from gensim.models import KeyedVectors

# 노드 기준 path 예시
# word2vec_path = os.getenv('HOME')+'aiffel/sentiment_classification/data/GoogleNews-vectors-negative300.bin.gz'
# 로컬 기준 path 예시
word2vec_path = '../data/sentiment_classification/GoogleNews-vectors-negative300.bin.gz'
word2vec = KeyedVectors.load_word2vec_format(word2vec_path, binary=True, limit=1000000)
vector = word2vec['computer']
vector  # 무려 300-dimension 짜리 워드 벡터..

결과는 노드에서 확인했던 shell 캡처로 대체함</br>
![Word2Vec dataset load a million counts](../images/8.png)</br>

In [ ]:
# 메모리를 다소 많이 소비하는 작업이니 유의해 주세요.
word2vec.similar_by_word("love")

단어들의 의미적 유사도가 가까운 것들을 확인하는 코드</br>

마찬가지로 노드에서 확인했던 shell 캡처로 대체함</br>
![cosine similarity within Word2vec](../images/9.png)</br>

아래 코드는 이전 스텝에서 학습했던 과정을 다시 기술한다.</br>
- 모델의 임베딩 레이어를 `Word2vec`의 것으로 대체한 예제 샘플
    - 모든 과정은 위의 것들과 마찬가지로 노드 확인결과 켬처본으로 대체예정

In [ ]:
vocab_size = 10000  # 어휘 사전의 크기입니다.(10,000개의 단어)
word_vector_dim = 300   # 워드 벡터의 차원수
embedding_matrix = np.random.rand(vocab_size, word_vector_dim)

# embedding_matrix에 워드 벡터를 단어별로 하나씩 차례차례 카피
for i in range(4,vocab_size):
    if index_to_word[i] in word2vec:
        embedding_matrix[i] = word2vec[index_to_word[i]]

In [ ]:
from tensorflow.keras.initializers import Constant

vocab_size = 10000      # 어휘 사전의 크기. 10,000개의 단어
word_vector_dim = 300   # 워드 벡터의 차원수

# 모델 구성
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, 
                                    word_vector_dim, 
                                    # 카피한 임베딩을 embedding_initializer로 활용
                                    embedding_initializer=Constant(emedding_matrix), 
                                    input_length=maxlen, 
                                    # trainable 값이 True면 Fine-tuning
                                    trainable=True))
model.add(tf.keras.layers.Conv1D(16, 7, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(5))
model.add(tf.keras.layers.Conv1D(16, 7, activation='relu'))
model.add(tf.keras.layers.GlobalMaxPooling1D())
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

![sequential model summarize list](../images/10.png)</br>

In [ ]:
# 학습의 진행
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

epochs = 20 # 훈련 횟수는 바꿔가면서 테스트해보자

history = model.fit(partial_x_train, partial_y_train, epochs=epochs, batch_size=512, validation_data=(x_val, y_val), verbose=1)

![learning process about 20 epochs case..](../images/11.png)</br>

In [ ]:
# 테스트셋을 통한 모델 평가
results = model.evaluate(x_test, y_test, verbose=2)

print(results)

![model evaluation for test set](../images/12.png)</br>